In [2]:
from urllib.request import urlopen
import bs4
import datetime as dt

def date_format(d):
    d = str(d).replace('-','.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    this_date = dt.date(yyyy,mm,dd)
    return this_date

def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
        
    naver_index = "http://finance.naver.com/sise/sise_index_day.nhn?code=" + index_cd+ '&page='+ str(page_n)
    
    source = urlopen(naver_index).read()    #지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')    #뷰티풀 수프로 태그별로 코드 분류
    
    dates=source.find_all('td',class_='date')     #<td class="date">태그에서 날짜 수집
    prices=source.find_all('td',class_='number_1')     #<td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            #날짜 처리
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                
                #종가 처리
                this_close = prices[n*4].text
                this_close = this_close.replace(',','')
                this_close = float(this_close)
            
                #딕셔너리에 저장
                historical_prices[this_date] = this_close
            elif this_date < start_date:
                return historical_prices
    
    #페이지네이션
        if last_page==0:
            last_page = source.find('td', class_='pgRR').find('a')['href']
            last_page = last_page.split('&')[1]
            last_page = last_page.split('=')[1]
            last_page = int(last_page)
        if page_n < last_page:
            page_n = page_n + 1
            historical_index_naver(index_cd, start_date, end_date, page_n, last_page)
    
        return historical_prices
    


In [4]:
index_cd = "KPI200"
historical_prices=dict()
historical_index_naver(index_cd, '2008-1-1', '2020-7-8')


{datetime.date(2020, 7, 2): 283.86,
 datetime.date(2020, 6, 24): 286.7,
 datetime.date(2020, 6, 16): 282.59,
 datetime.date(2020, 6, 8): 290.77,
 datetime.date(2020, 5, 29): 268.32,
 datetime.date(2020, 5, 21): 263.74,
 datetime.date(2020, 5, 13): 255.85,
 datetime.date(2020, 5, 4): 250.6,
 datetime.date(2020, 4, 22): 251.88,
 datetime.date(2020, 4, 13): 243.4,
 datetime.date(2020, 4, 3): 231.7,
 datetime.date(2020, 3, 26): 229.34,
 datetime.date(2020, 3, 18): 215.83,
 datetime.date(2020, 3, 10): 264.67,
 datetime.date(2020, 3, 2): 270.37,
 datetime.date(2020, 2, 21): 292.42,
 datetime.date(2020, 2, 13): 300.93,
 datetime.date(2020, 2, 5): 292.02,
 datetime.date(2020, 1, 28): 292.77,
 datetime.date(2020, 1, 16): 302.78,
 datetime.date(2020, 1, 8): 289.42,
 datetime.date(2019, 12, 27): 295.2,
 datetime.date(2019, 12, 18): 294.22,
 datetime.date(2019, 12, 10): 279.31,
 datetime.date(2019, 12, 2): 277.44,
 datetime.date(2019, 11, 22): 279.43,
 datetime.date(2019, 11, 14): 283.93,
 datetim